<a href="https://colab.research.google.com/github/maxideluca277/maximodeluca_projects/blob/main/simulador_ensamblador.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
class CPU:

    def __init__(self):
        # Diccionario de registros simulados (32 bits)
        self.registros = {
            'eax': 0,
            'ebx': 0,
            'ecx': 0,
            'edx': 0
        }

        # Diccionario de banderas (flags)
        self.flags = {
            'carry': 0,
            'zero': 0,
            'sign': 0,
            'overflow': 0
        }

    def actualizar_flags(self, resultado, anterior=None, operando=None, tipo=None):
        """Actualiza los flags Z, S, C y V según el resultado y la operación"""

        # ZF = 1 si el resultado es 0
        self.flags['zero'] = int(resultado == 0)

        # SF = 1 si el bit más significativo del resultado (bit 31) es 1
        self.flags['sign'] = int((resultado >> 31) & 1)

        # ADD: calcula carry (sin signo) y overflow (con signo)
        if tipo == 'add' and anterior is not None and operando is not None:
            suma_completa = anterior + operando   # suma antes de truncar a 32 bits

            # CF = 1 si la suma excedió 32 bits (es decir, mayor que 0xFFFFFFFF)
            self.flags['carry'] = int(suma_completa > 0xFFFFFFFF)

            # Overflow en suma con signo: si los operandos tienen el mismo signo
            # y el resultado tiene signo distinto al de los operandos.
            r_sign = (resultado >> 31) & 1
            a_sign = (anterior >> 31) & 1
            b_sign = (operando >> 31) & 1
            self.flags['overflow'] = int((a_sign == b_sign) and (r_sign != a_sign))

        # INC: afecta ZF, SF y OF, pero NO CF (se deja como estaba)
        elif tipo == 'inc' and anterior is not None:
            self.flags['overflow'] = int(anterior == 0x7FFFFFFF)

        # DEC: afecta ZF, SF y OF, pero NO CF (se deja como estaba)
        elif tipo == 'dec' and anterior is not None:
            self.flags['overflow'] = int(anterior == 0x80000000)

        # SHL y SHR: ya calculan CF en la instrucción, acá solo fijamos OF=0
        elif tipo in ('shl', 'shr'):
            self.flags['overflow'] = 0

        # MOV y otros: en x86 real no modifican flags → no tocar carry ni overflow
        elif tipo == 'mov':
            pass

    def ejecutarInstrucciones(self, instruccion: str):
        # Parsear instrucción
        partesInstruccion = instruccion.lower().replace(',', ' ').split()
        if len(partesInstruccion) < 2:
            raise ValueError("Error. Debe introducir por lo menos 2 parámetros")

        operacion = partesInstruccion[0]
        destino = partesInstruccion[1]
        fuente = None
        if len(partesInstruccion) > 2:
            fuente = partesInstruccion[2]

        if destino not in self.registros:
            raise ValueError(f"Destino inválido: {destino}")

        # Resolver valor de fuente (puede ser registro o inmediato)
        if fuente is None:
            valorFuente = None
        elif fuente in self.registros:
            valorFuente = self.registros[fuente]
        else:
            try:
                valorFuente = int(fuente, 0)  # admite decimal, binario (0b) o hex (0x)
            except ValueError:
                raise ValueError(f"Fuente inválida: {fuente}")

        # Ejecutar operación
        match operacion:

            case 'mov':
                if valorFuente is None:
                    raise ValueError("mov requiere una fuente, tercer parámetro")
                self.registros[destino] = valorFuente & 0xFFFFFFFF
                # En x86 real MOV no toca flags, así que no llamamos a actualizar_flags

            case 'add':
                if valorFuente is None:
                    raise ValueError("add requiere un valor fuente")
                original = self.registros[destino]
                suma_completa = original + valorFuente
                # Guardamos solo los 32 bits menos significativos
                self.registros[destino] = suma_completa & 0xFFFFFFFF
                self.actualizar_flags(self.registros[destino], anterior=original, operando=valorFuente, tipo='add')

            case 'inc':
                original = self.registros[destino]
                self.registros[destino] = (original + 1) & 0xFFFFFFFF
                self.actualizar_flags(self.registros[destino], anterior=original, operando=1, tipo='inc')

            case 'dec':
                original = self.registros[destino]
                self.registros[destino] = (original - 1) & 0xFFFFFFFF
                self.actualizar_flags(self.registros[destino], anterior=original, operando=1, tipo='dec')

            case 'shl':  # desplazamiento a la izquierda
                if valorFuente is None:
                    raise ValueError("shl requiere una fuente")
                if valorFuente <= 0 or valorFuente > 32:
                    raise ValueError("El desplazamiento debe ser entre 1 y 32")

                original = self.registros[destino]

                # CF = último bit que "sale" (el bit 31 movido hacia afuera)
                self.flags["carry"] = (original >> (32 - valorFuente)) & 1

                # Desplazamos a la izquierda y nos quedamos con los 32 bits menos significativos
                self.registros[destino] = (original << valorFuente) & 0xFFFFFFFF

                self.actualizar_flags(self.registros[destino], tipo='shl')

            case 'shr':  # desplazamiento a la derecha
                if valorFuente is None:
                    raise ValueError("shr requiere una fuente")
                if valorFuente <= 0 or valorFuente > 32:
                    raise ValueError("El desplazamiento debe ser entre 1 y 32")

                original = self.registros[destino]

                # CF = último bit que "sale" (el menos significativo antes del shift)
                self.flags["carry"] = (original >> (valorFuente - 1)) & 1

                # Desplazamos a la derecha y nos quedamos con los 32 bits menos significativos
                self.registros[destino] = (original >> valorFuente) & 0xFFFFFFFF

                self.actualizar_flags(self.registros[destino], tipo='shr')

            case 'xchg':  # intercambio de registros
                if fuente is None:
                    raise ValueError("xchg requiere una fuente")
                if destino not in self.registros or fuente not in self.registros:
                    raise ValueError("xchg solo está implementado entre registros")
                # Intercambiamos valores
                self.registros[destino], self.registros[fuente] = self.registros[fuente], self.registros[destino]
                # En x86 real, XCHG no modifica flags

            case _:
                print(f"Instrucción no reconocida: {operacion}")

    def mostrarRegistros(self):
        print("\n📊 REGISTROS (en hexadecimal):")
        for reg, val in self.registros.items():
            print(f"{reg}: {hex(val)}")

    def mostrarFlags(self):
        print("\n📊 FLAGS :")
        for reg, val in self.flags.items():
            print(f"{reg}: {bin(val)}")


cpu = CPU()

print("Ingrese instrucciones tipo assembler (ej: 'mov eax, 5')")
print("Escriba 'fin' para terminar\n")

while True:
    entrada = input(">> ").strip()
    if entrada.lower() == 'fin':
        break
    if entrada == "":
        continue
    try:
        cpu.ejecutarInstrucciones(entrada)
        cpu.mostrarRegistros()
        cpu.mostrarFlags()
    except Exception as e:
        print(f"⚠️  Error: {e}")


Ingrese instrucciones tipo assembler (ej: 'mov eax, 5')
Escriba 'fin' para terminar

>> fin
